In [2]:
import os, sys
from glob import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from torchvision.models import alexnet

from tqdm import tqdm
import numpy as np

import cv2 as cv

In [3]:
print(os.getcwd())

D:\GitCloneProject\CE-AB-CLS\OtherSOTA


In [4]:
os.chdir("..")

In [5]:
data_dir = os.getcwd() + "\\Data set\\Original Form"
print(os.listdir(data_dir))

['Test', 'Train']


In [36]:
class CEDataset(Dataset):
    def __init__(self, root_dir = data_dir, subset = "Train"):
        self.root_dir = root_dir + f"\\{subset}"
        self.img_paths = glob(self.root_dir + "\\*\\*")
        self.transform = A.Compose([
            # A.RandomCrop(height = 256, width = 256),
            A.ColorJitter(p=0.5),
            A.GaussianBlur(p=0.5),
            A.RandomShadow(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.Resize(height = 256, width = 256, p = 1),
        ])
        super().__init__()
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        path = self.img_paths[idx]
        img = cv.imread(path)
        
        if "Normal" in path:
            label = torch.tensor(0)
        elif "Abnormal" in path:
            label = torch.tensor(1)
        else:
            raise Exception("not shown type of label in path")
        
        return torch.from_numpy(self.transform(image = img)["image"]/255).permute(-1, 0, 1), label  

In [37]:
train_dataset = CEDataset(subset = "Train")
test_dataset = CEDataset(subset = "Test")

print(len(train_dataset))
print(len(test_dataset))

176
176


In [38]:
sample_img, sample_label = train_dataset[0]

In [39]:
print(train_dataset.img_paths[0])
print(test_dataset.img_paths[0])

D:\GitCloneProject\CE-AB-CLS\Data set\Original Form\Train\Group 1 - Normal\CHGastro_Normal_001.png
D:\GitCloneProject\CE-AB-CLS\Data set\Original Form\Test\Group 1 - Normal\CHGastro_Normal_002.png


In [45]:
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = 32)
test_dataloader = DataLoader(test_dataset, shuffle = False, batch_size = 1)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = alexnet(weights = 'IMAGENET1K_V1').to(device)
model.fc = nn.Linear(1000, 2)
# model.sf = nn.Softmax(2)

In [22]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [44]:
size = len(train_dataloader)
model.train()
epochs = 100
for t in range(epochs):
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device, dtype = torch.float), y.to(device)        

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # accuracy
        correct = (pred.argmax(1) == y).type(torch.float).sum().item()
        acc = correct / y.shape[0]

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] acc: {acc}")

loss: 0.612485  [   32/    6] acc: 0.65625
loss: 0.572555  [   32/    6] acc: 0.65625
loss: 0.527337  [   32/    6] acc: 0.75
loss: 0.567612  [   32/    6] acc: 0.71875
loss: 0.606250  [   32/    6] acc: 0.65625
loss: 0.671754  [   32/    6] acc: 0.53125
loss: 0.680170  [   32/    6] acc: 0.625
loss: 0.533879  [   32/    6] acc: 0.71875
loss: 0.554774  [   32/    6] acc: 0.71875
loss: 0.514703  [   32/    6] acc: 0.75
loss: 0.612036  [   32/    6] acc: 0.625
loss: 0.488404  [   32/    6] acc: 0.71875
loss: 0.509546  [   32/    6] acc: 0.71875
loss: 0.503870  [   32/    6] acc: 0.71875
loss: 0.562823  [   32/    6] acc: 0.625
loss: 0.473390  [   32/    6] acc: 0.84375
loss: 0.453033  [   32/    6] acc: 0.78125
loss: 0.487992  [   32/    6] acc: 0.78125
loss: 0.877947  [   32/    6] acc: 0.34375
loss: 0.591797  [   32/    6] acc: 0.65625
loss: 0.568401  [   32/    6] acc: 0.71875
loss: 0.496526  [   32/    6] acc: 0.8125
loss: 0.534080  [   32/    6] acc: 0.6875
loss: 0.388145  [   32/  

In [47]:
pred = []
true = []
for batch, (X, y) in enumerate(test_dataloader):
    X, y = X.to(device, dtype = torch.float), y.to(device)
    
    pred.append(model(X).argmax(1).item())
    true.append(y.item())

In [48]:
print(len(pred))
print(len(true))

176
176


In [49]:
from sklearn.metrics import classification_report

print(classification_report(true, pred, target_names=["Normal", "Abnormal"]))

              precision    recall  f1-score   support

      Normal       0.54      0.52      0.53        56
    Abnormal       0.78      0.79      0.79       120

    accuracy                           0.70       176
   macro avg       0.66      0.65      0.66       176
weighted avg       0.70      0.70      0.70       176

